In [ ]:
# default_exp experimental

In [ ]:
#hide
from IPython.display import clear_output
from nbdev.export import notebook2script
%reload_ext autoreload
%autoreload 2

In [ ]:
#export
import json
import logging

logger = logging.getLogger()
logger.setLevel("INFO")

# experimental

> description

In [ ]:
def str2tensor(s):
    lines = s.split("\n")
    tuples = [l.split("\t") for l in lines]
    tuples = [[int(e) for e in t] for t in tuples]
    return torch.tensor(tuples, dtype=torch.float32)

In [ ]:
#server
from practical_ai.vision.gan.core import GAN